In [167]:
import pyvista as pv
import numpy as np
import math  
import trame

# import trimesh
# import pyvox

# From CAD to VOXEL

In [168]:
Path = 'Object\sphere.stl'
mesh = pv.read(Path)

x_min, x_max, y_min, y_max, z_min, z_max = mesh.bounds

scale_print= max(x_max - x_min,y_max - y_min,z_max - z_min)
# print(mesh.length)  diagonal
scale_voxel=0.1

voxels = pv.voxelize(mesh, density=scale_voxel, check_surface=False)

surface = voxels.extract_surface()
surface_points = surface.points
surface_points -= np.array([x_min,y_min,z_min]) 
surface_points = np.divide(surface_points, scale_voxel).astype(int)
surface_points

pyvista_ndarray([[ 7, 11,  0],
                 [ 7, 12,  0],
                 [ 7, 12,  1],
                 ...,
                 [19, 12, 25],
                 [19, 13, 25],
                 [19, 14, 25]])

# VOXEL model

In [169]:
class Voxels():
    
    def __init__(self, scale_print, scale_voxel):
        
        self.scale_print = scale_print
        self.scale_voxel = scale_voxel

        self.nb_voxels = math.ceil(scale_print/scale_voxel)
        
        self.voxels = np.zeros((self.nb_voxels,self.nb_voxels,self.nb_voxels))

    def Voxelise_object(self,object):
        Path = object
        mesh = pv.read(Path)

        x_min, x_max, y_min, y_max, z_min, z_max = mesh.bounds

        voxels = pv.voxelize(mesh, density=self.scale_voxel, check_surface=False)

        surface = voxels.extract_surface()
        surface_points = surface.points
        surface_points -= np.array([x_min,y_min,z_min]) 
        surface_points = np.divide(surface_points, scale_voxel).astype(int)
        return surface_points

    def add_density(self, coord):

        self.voxels[coord[:,0],coord[:,1],coord[:,2]]=1

    def substracted_density(self,coord):
        
        self.voxels[coord[:,0],coord[:,1],coord[:,2]]=0

    def density(self, coord):
        density_bool = self.voxels[coord[:,0],coord[:,1],coord[:,2]]==1
        return not np.any(~density_bool)
    

    def move_object_translation(self, object, position_ini, position_fin, scale=True):  
        # scale : In the good referential
        object = object.astype(np.float64)
        object += (position_fin - position_ini)*(self.scale_voxel*(1-scale)+scale)

        if  not scale :          # correction move to be affected at the good voxel
            return np.floor(object).astype(int)
        return object.astype(int)
    

    def rotation(self, object, theta, phi):

        z=np.array([0,0,1])
        # Z -> x vers y
        rot_theta = np.array([[np.cos(theta), -np.sin(theta), 0],
                            [np.sin(theta), np.cos(theta), 0],
                            [0, 0, 1]])

        object_rot = rot_theta @ object.T

        x_prime = rot_theta @ np.array([1, 0, 0]).T
        y_prime = np.cross(z, x_prime.T)

        # Y' -> z vers x'
        I=np.eye(3)
        y_x=y_prime[0]
        y_y=y_prime[1]
        y_z=y_prime[2]
        P=np.array([ [ y_x*y_x, y_x*y_y, y_x*y_z],
                    [ y_x*y_y, y_y*y_y, y_y*y_z],
                    [ y_x*y_z , y_y*y_z, y_z*y_z]])
        
        Q=np.array([[0,-y_z,y_y],
                    [y_z,0,-y_x],
                    [-y_y, y_x,0]])
        
        Rot = P + np.cos(phi)*(I-P)+np.sin(phi)*Q

        object_rot = Rot @ object_rot

        return object_rot.T.astype(int)
        
       



In [170]:
model=Voxels(scale_print,scale_voxel)
model.voxels.shape

(26, 26, 26)

In [171]:
model.add_density(surface_points)
model.density(surface_points)

True

[25,25,25] ne respecte pas les condition d'appartenance 

In [172]:
p = pv.Plotter()
p.add_mesh(model.voxels, color=True, show_edges=True, opacity=0.5)
p.add_mesh(object, color=True, show_edges=True, opacity=0.8)
p.add_mesh(np.array([24,1,0]), color=True, show_edges=True, opacity=0.8)
p.show()


c:\Users\sdesg\Documents\Voxel_py310\myenv\lib\site-packages\pyvista\utilities\helpers.py:507: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


Widget(value="<iframe src='http://localhost:58478/index.html?ui=P_0x20852cf32e0_74&reconnect=auto' style='widt…

# Change of position

- Translation test

In [173]:
object = surface_points
position_ini=np.array([ 7, 11,  0])
position_fin=np.array([0, 0, 0])
object

pyvista_ndarray([[ 7, 11,  0],
                 [ 7, 12,  0],
                 [ 7, 12,  1],
                 ...,
                 [19, 12, 25],
                 [19, 13, 25],
                 [19, 14, 25]])

In [174]:
object=model.move_object_translation(object,position_ini,position_fin)

- Rotation test

In [177]:
theta = np.radians(0.)  # x vers y
phi = np.radians(90)   # z vers x'
# le cas ou theta egale à 0 ou 90 ou 180 ou ... l'axe de rotation devient x ou y 
# et donc les points bouges pas
point = np.array([[0, 0, 1],
                  [0, 1, 0],
                  [1, 0, 0]])
model.rotation(point, theta, phi)

array([[ 1,  0,  0],
       [ 0,  1,  0],
       [ 0,  0, -1]])

- Vector Creation 

In [178]:
def Vector_Creation(theta, angle_max):
    qotient = int(angle_max // theta) + 1
    Circle = {i: [] for i in range(qotient)}
    Circle[0]=np.array([0])
    for i in range(1,qotient):
        u=np.cos(theta)
        v=np.cos(i*theta)
        v=v*v
        sigma=np.arccos((u-v)/(1-v))
        qotient = int(2*np.pi // sigma) + 1
        Circle[i] = np.array([ k*sigma for k in range(qotient)])
    return Circle

In [179]:
theta=np.pi / 6
angle_max=np.pi / 2
circle=Vector_Creation(theta, angle_max)
point=np.array([0,0,1])
point_rotate=[]
# for k in circle:
for ele in circle[3]:
        point_rotate.append(model.rotation(point,ele, 3*theta))
point_rotate=np.array(point_rotate)
point_rotate
p = pv.Plotter()
p.add_mesh(point_rotate, color=True, show_edges=True, opacity=0.5)
p.show()


c:\Users\sdesg\Documents\Voxel_py310\myenv\lib\site-packages\pyvista\utilities\helpers.py:507: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


Widget(value="<iframe src='http://localhost:58478/index.html?ui=P_0x208389accd0_75&reconnect=auto' style='widt…

idée : pour chaque vecteur position créée determiner tous en meme temps leur faisabilité (pour tous les points / étages)

In [ ]:
# stacker toutes les positions pour chaque points réussir a le multiplier par la matrice rotation et vérifier s'il y a densité
# conserver la forme mesh de l'object pendant sa rotation est voxéliser l'object au moment ou il est en position de test 
# pour chaque points l'inclinaison de la machin est la meme il suffit donc seulement d'en effectuer une seul de translater l'object sur toutes 
# les positions et de vérifier s'il est bien

- centrer l'object en 0 pour XY

In [180]:
mesh = pv.read('Object\Extruder.stl')
center = mesh.center
center[-1]=0.
mesh.points -= center

- modifier l'echelle de l'object

In [181]:
# modifier l'echelle de l'object
scale_factor = 0.25/2   # multiplie la taille par 2
mesh_scale = mesh.scale([0.25,0.25,0.25], inplace=False)
pl = pv.Plotter(shape=(1, 2))
pl.subplot(0, 0)
pl.show_axes()
_ = pl.show_grid()
_ = pl.add_mesh(mesh)
pl.subplot(0, 1)
pl.show_axes()
_ = pl.show_grid()
_ = pl.add_mesh(mesh_scale)
pl.show()

Widget(value="<iframe src='http://localhost:58478/index.html?ui=P_0x20866de6620_76&reconnect=auto' style='widt…

In [182]:
def rotation_mesh(mesh, theta, phi):
    mesh_copy = mesh.copy()
    z=np.array([0,0,1])
    mesh_copy.rotate_vector(z, theta, inplace=True)
    # Z -> x vers y
    
    rot_theta = np.array([[np.cos(theta), -np.sin(theta), 0],
                        [np.sin(theta), np.cos(theta), 0],
                        [0, 0, 1]])

    x_prime = rot_theta @ np.array([1, 0, 0]).T
    y_prime = np.cross(z, x_prime.T)
    mesh_copy.rotate_vector(y_prime, phi, inplace=True)
    return mesh_copy

In [183]:
#rotation 
theta = 45
phi = 45
mesh_rotation = rotation_mesh(mesh_scale,theta, phi)   #deg
pl = pv.Plotter(shape=(1, 2))
pl.subplot(0, 0)
pl.show_axes()
_ = pl.show_grid()
_ = pl.add_mesh(mesh_scale)
_ = pl.add_axes_at_origin()
pl.subplot(0, 1)
pl.show_axes()
_ = pl.show_grid()
_ = pl.add_mesh(mesh_rotation)
_ = pl.add_axes_at_origin()
pl.show()

Widget(value="<iframe src='http://localhost:58478/index.html?ui=P_0x2084d032650_77&reconnect=auto' style='widt…

# Dijkstra's shortest path algorithm

In [184]:
import networkx as nx

In [185]:
G = nx.DiGraph()  # noeud orienté
G.add_weighted_edges_from([("A", "C",3),
                           ("A", "E",2),
                           ("A", "D",1),
                           ("D", "C",1)])

vectors = {
    "A": [1, 2, 3],
    "D": [4, 5, 6],
    "C": [7, 8, 9]
}
for sommet, vecteur in vectors.items():
    G.nodes[sommet]["vecteur"] = vecteur
shortest_path = nx.dijkstra_path(G, "A", "C")
shortest_path

['A', 'D', 'C']

# Free Path

In [186]:
def  FreePath(vector_head, tool_path, theta, theta_max, head_print_CAD, printed_object_CAD, scale_voxel, scale_print):

    vox = Voxels(scale_print,scale_voxel)

    surface_printed = vox.Voxelise_object(printed_object_CAD)
    vox.add_density(surface_printed)

    print_head = vox.Voxelise_object(head_print_CAD)


#   build_graph
    num_layers = len(tool_path)
    source = 's'
    target = 't'
    
    G = nx.DiGraph()
    G.add_node(source)
    G.add_node(target)
                
    layer_vertex = {i: [] for i in range(num_layers)}
    Circle=Vector_Creation(theta, angle_max)
    
    for i in range(num_layers):
        if i > 0:
            prev_layer = i - 1    # Récupérez les sommets de l'étage précédent
            prev_layer_vertices = layer_vertex[prev_layer]

        for j, vec1 in enumerate(Circle):
            position = move_object_rotation(print_head,tool_path[i], angle =(Circle[-2], Circle[-1])) # modifie la position de la tete dans les coordonnées associé au voxels
            if not vox.density(position): # s'il n'y a pas de desnité (pas d'obstacle)
                current_vertex = (i, j)  #  name of the vertex
                G.add_node(current_vertex, vector=vec1, angle=theta) # creer le sommet white atribute 
                layer_vertex[i].append(current_vertex)  # Ajoute les sommets aux listes d'étages
                if i > 0:
                    for prev_vertex in prev_layer_vertices: # creer des liaison entre les etages si la condition angulaire est respécté
                        angle = abs(G.nodes[prev_vertex]['angle']-G.nodes[current_vertex]['angle'])
                        if angle <= theta:
                            G.add_edge(prev_vertex, current_vertex, weight=angle)
                if i == 0:
                    G.add_edge(source, current_vertex, weight=0)
                if i == num_layers-1:
                    G.add_edge(current_vertex,target ,weight=0)
        vox.add_density(tool_path[i])

    shortest_path = nx.dijkstra_path(G, source, target)


    